In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import importlib
import gc
import io
import os
from itertools import combinations

from IPython.display import display
pd.set_option('display.max_columns', 99)
pd.set_option('display.max_rows', 200)

pd.reset_option('display.float_format')
pd.set_option('display.max_colwidth', None)  

from sitecustomize import ROOT # lib này được khởi tạo ban đầu dự án

import helpers.view as view
import helpers.EDA as EDA

importlib.reload(view)
importlib.reload(EDA)

<module 'helpers.EDA' from 'd:\\Data Science\\helpers\\EDA.py'>

In [35]:
def cache_clear():
    for var in list(globals()):  
        if var not in _keep_vars and not var.startswith("_"):  
            del globals()[var]  

    gc.collect()
_keep_vars = set(globals().keys())  # lưu biến gốc

# POS CASH balance (số dư khoản vay tiêu dùng - point of sale (cash loans ; comsumer loans))

In [45]:
pos_balance = pd.read_pickle(ROOT + "/data/pkl/pos_cash_balance.p")

# feature engineering

In [46]:
pos_balance.loc[((pos_balance["NAME_CONTRACT_STATUS"]=='Active') & (pos_balance["CNT_INSTALMENT_FUTURE"]==0)), 'NAME_CONTRACT_STATUS'] = 'Completed' ####### sửa
pos_balance.loc[(["NAME_CONTRACT_STATUS"]=='Completed') & (pos_balance["CNT_INSTALMENT_FUTURE"]!=0), "CNT_INSTALMENT_FUTURE"] = "Active" ####### sửa

C:\Users\Admin\AppData\Local\Temp\ipykernel_24608\1150525613.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Active' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  pos_balance.loc[(["NAME_CONTRACT_STATUS"]=='Completed') & (pos_balance["CNT_INSTALMENT_FUTURE"]!=0), "CNT_INSTALMENT_FUTURE"] = "Active" ####### sửa


In [47]:
pos_balance_0 = pos_balance[pos_balance['CNT_INSTALMENT_FUTURE']==0].copy()
pos_balance_1 = pos_balance[pos_balance['CNT_INSTALMENT_FUTURE']>0].copy()

In [48]:
pos_balance_0.sort_values(['SK_ID_PREV', 'MONTHS_BALANCE'], ascending=[True, False], inplace=True)

In [49]:
pos_balance_0.drop_duplicates(['SK_ID_PREV', "NAME_CONTRACT_STATUS"], keep='last', inplace=True)

In [50]:
pos_balance = pd.concat([pos_balance_0, pos_balance_1], ignore_index=True)

In [51]:
pos_balance['CNT_INSTALMENT-s-CNT_INSTALMENT_FUTURE'] = pos_balance['CNT_INSTALMENT'] - pos_balance['CNT_INSTALMENT_FUTURE']
pos_balance['CNT_INSTALMENT_FUTURE-d-CNT_INSTALMENT'] = pos_balance['CNT_INSTALMENT_FUTURE'] / pos_balance['CNT_INSTALMENT']

In [52]:
pos_balance.sort_values(['SK_ID_PREV', 'MONTHS_BALANCE'], inplace=True)
pos_balance.reset_index(drop=True, inplace=True)

In [53]:
_keep_vars.update(["pos_balance"])

In [54]:
cache_clear()